In [73]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import re

In [34]:
# Initialization

load_dotenv(override=True)

open_router_api_key = os.getenv('OPEN_ROUTER_API_KEY')
Calendarific_api_key = os.getenv('CALENDARIFIC_API_KEY')

if not open_router_api_key or not Calendarific_api_key:
    raise ValueError("API keys missing. Please ensure OPEN_ROUTER_API_KEY and CALENDARIFIC_API_KEY are set in your .env file.")

    
MODEL = "nvidia/nemotron-3-nano-30b-a3b:free"

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key= open_router_api_key,
)


In [76]:
SYSTEM_MESSAGE = """You are a helpful Public Holiday Assistant And your primary communication language is English (You can change it if the user request it). 

When users ask about holidays, extract:
1. The country name
2. The date (in YYYY-MM-DD format) or month/year

Then respond in this exact format:
QUERY: [check_holiday|get_month] | country_name | date_or_month_year

Examples:
- User: "Is Jan 26 2025 a holiday in India?"
  You: QUERY: check_holiday | India | 2025-01-26

- User: "Show me all holidays in USA for March 2025"
  You: QUERY: get_month | USA | 2025-03

If the user's message is unclear or just chatting, respond normally without the QUERY format.
"""

In [52]:
def get_country_code(country_name):
    mapping = {
        "india": "IN", "united states": "US", "usa": "US", "us": "US",
        "uk": "GB", "united kingdom": "GB", "britain": "GB",
        "canada": "CA", "australia": "AU", "germany": "DE", 
        "france": "FR", "japan": "JP", "china": "CN", 
        "brazil": "BR", "mexico": "MX", "italy": "IT", "spain": "ES"
    }
    return mapping.get(country_name.lower().strip())

def fetch_holidays_from_api(country_code, year, month=None, day=None):
    url = "https://calendarific.com/api/v2/holidays"
    params = {
        "api_key": Calendarific_api_key,
        "country": country_code,
        "year": year,
        "type": "national"
    }
    if month: params["month"] = month
    if day: params["day"] = day
    
    try:
        response = requests.get(url, params=params, timeout=10)
        data = response.json()
        print("Data: ",data)
        if data.get("meta", {}).get("code") == 200:
            return data.get("response", {}).get("holidays", [])
    except Exception as e:
        print(f"API Error: {e}")
    return []

In [53]:
def check_public_holiday(country, date_str):
    """Check if a specific date is a holiday."""
    code = get_country_code(country)
    if not code: 
        return f"Sorry, I don't have data for '{country}'. Try: India, USA, UK, Canada, Australia, etc."

    try:
        dt = datetime.strptime(date_str, "%Y-%m-%d").date()
    except ValueError:
        return "Invalid date format. Please use YYYY-MM-DD."

    holidays = fetch_holidays_from_api(code, dt.year, dt.month, dt.day)
    
    status = "today" if dt == date.today() else ("in the past" if dt < date.today() else "upcoming")
    
    if holidays:
        details = [f"**{h['name']}**" for h in holidays]
        return f"✅ Yes! {date_str} is a public holiday in {country}:\n" + "\n".join(details) + f"\n\n({status})"
    else:
        month_holidays = fetch_holidays_from_api(code, dt.year, dt.month)
        upcoming = [
            f"• {h['date']['iso']}: {h['name']}" 
            for h in month_holidays 
            if h.get('date',{}).get('iso') > date_str 
        ]
        
        msg = f"❌ No, {date_str} is not a public holiday in {country}."
        if upcoming:
            msg += f"\n\n📅 Upcoming holidays in {dt.strftime('%B %Y')}:\n" + "\n".join(upcoming[:5])
        else:
            msg += f"\n\nNo more holidays recorded for {dt.strftime('%B %Y')}."
        return msg



In [54]:
def get_month_holidays(country, year, month):
    """Get all holidays for a month."""
    code = get_country_code(country)
    if not code: 
        return f"Sorry, I don't have data for '{country}'."
    
    holidays = fetch_holidays_from_api(code, year, month)
    if not holidays: 
        return f"No public holidays found for {country} in {month}/{year}."
    
    month_name = datetime(year, month, 1).strftime('%B %Y')
    result = f"📅 Public holidays in {country} for {month_name}:\n\n"
    result += "\n".join([f"• {h['date']['iso']}: **{h['name']}**" for h in holidays])
    return result


In [59]:
# There's a particular dictionary structure that's required to describe our function:

function_description = {
    "name": "check_public_holiday",
    "description": "Check if a specific date is a public holiday.",
    "parameters": {
        "type": "object",
        "properties": {
            "country": {
                "type": "string",
                "description": "The country that the user want to check for public holidays.",
            },
            "date": {
                "type": "string",
                "description": "The date that the user want to check for public holidays.",
            }
        },
        "required": ["country"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": function_description}]
tools

In [70]:
def parse_query(text):
    """Parse the model's QUERY response"""
    match = re.search(r'QUERY:\s*(\w+)\s*\|\s*([^|]+)\s*\|\s*(.+)', text)
    if match:
        query_type = match.group(1).strip()
        country = match.group(2).strip()
        date_info = match.group(3).strip()
        return query_type, country, date_info
    return None, None, None

In [75]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": SYSTEM_MESSAGE}] + history + [{"role": "user", "content": message}]
    response = client.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    print(response)
    llm_response = response.choices[0].message.content
    query_type, country, date_info = parse_query(llm_response)

    if query_type == "check_holiday":
        return check_public_holiday(country, date_info)
    elif query_type == "get_month":
        # Parse year-month
        try:
            year, month = date_info.split('-')
            return get_month_holidays(country, int(year), int(month))
        except:
            return "Sorry, I couldn't parse the date. Please specify like 'March 2025' or '2025-03'."
    else:
        # Regular conversation
        return llm_response


gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7879
* To create a public link, set `share=True` in `launch()`.
